In [19]:
# Install the package
import numpy as np
from sympy import *
from scipy.optimize import fsolve


In [20]:
var('z')

grid = .001
nobs = int(1/grid)+1

In [21]:
# fixed exogenous parameters
successes = 10
pbar = .49
pi_m = 200 
theta_m = 0
xibar = .005
alphabar = .05
beta = .8
eta = 10000
epsilon = .6

In [28]:
# derived values
xbar = successes / eta
sigma = 1 + 1/epsilon
gamma = pbar/sigma/xbar**(sigma-1)
mu_m = pi_m/2
pi_c = pi_m*(1+xibar)
mu_c = pi_c/2
print('xbar: ',xbar)
print('sigma: ',sigma)
print('gamma: ',gamma)
print('mu_m: ',mu_m)
print('pi_c: ',pi_c)
print('mu_c: ',mu_c)

xbar:  0.001
sigma:  2.666666666666667
gamma:  18375.000000000033
mu_m:  100.0
pi_c:  200.99999999999997
mu_c:  100.49999999999999


In [27]:
#begin psibar
pi_s_0 = pi_m*( 1 + 2/9*z*( z*(1-exp(-3./z)) - 3) )
theta_s_0 = pi_m*( 2*z/9 + exp(-3./z)/18* ( 9*z**2 + 32*z + 60 ) )
p0 = (1-beta)*( (1-alphabar)*(pi_c-pi_m) +alphabar*(pi_m-pi_s_0 ) ) \
    + beta*( (1-alphabar)*theta_m + alphabar*theta_s_0 )
psibar = fsolve(lambdify(z,p0-pbar),1)[0]
psibar

0.09779483275875955

In [24]:
#end psibar
pi_s = pi_m*( 1 + 2/9*psibar*( psibar*(1-exp(-3./psibar)) - 3) )
theta_s = pi_m*( 2*psibar/9 + exp(-3./psibar)/18* ( 9*psibar**2 + 32*psibar + 60 ) )
mu_s = pi_m*(1/2 + psibar/9*(3+psibar) + exp(-3/psibar)*psibar*(5*psibar + 12)/36)
#print the outcome
print('pi_s: ',pi_s)
print('theta_s: ',theta_s)
print('mu_s: ',mu_s)

pi_s:  187.385748046135
theta_s:  4.34643701153384
mu_s:  106.732185057569


In [26]:
#begin alpha_h
p = (1-beta)*( (1-z)*(pi_c-pi_m) +z*(pi_m-pi_s) ) \
    + beta*( (1-z)*theta_m + z*theta_s )
prize = p
x0 = (prize/gamma/sigma)**(1/(sigma-1))
w0 = z*mu_m + (1-z)*mu_c
a0 = (1-x0)*mu_m + x0*w0 - mu_m
prize = z*theta_s+(1-z)*theta_m
x1 = (prize/gamma/sigma)**(1/(sigma-1))
w1 = z*mu_s + (1-z)*mu_m
a1 = (1-x1)*mu_m + x1*w1 - mu_m 
alpha_h = float( (mu_c-mu_m) / ( (mu_c-mu_m) + (mu_s-mu_m) ) )
alpha_h 

0.06913539905573864

In [29]:
alpha_p = .5
f1=open('basecase.tex', 'w+')
f2=open('analysis.csv', 'w+')
print('alpha,x0,w0,a0,x1,w1,a1',file=f2)

In [30]:
v_n = v_h = v_p = v_o = v_t = 0
x_n = x_h = x_p = x_o = x_t = 0
c_n = c_h = c_p = c_o = c_t = 0
oa_n = oa_h = oa_p = oa_o = oa_t = 0
aa0 = np.zeros(nobs)
aa1 = np.zeros(nobs)
flag = test = 0
alphastar = 0

In [31]:
for i in range(0,nobs):
    alpha = i*grid
    dens = np.exp(-alpha/alphabar)/alphabar*grid
    test += dens
    gain_inc = (1-alpha)*(pi_c-pi_m) +alpha*(pi_m-pi_s)
    gain_sta = (1-alpha)*theta_m + alpha*theta_s
    p = (1-beta)*( (1-alpha)*(pi_c-pi_m) +alpha*(pi_m-pi_s) ) \
        + beta*( (1-alpha)*theta_m + alpha*theta_s )
    prize0 = p
    prize1 = alpha*theta_s+(1-alpha)*theta_m
    x0 = (prize0/gamma/sigma)**(1/(sigma-1))
    x1 = (prize1/gamma/sigma)**(1/(sigma-1))
    w0 = alpha*mu_m + (1-alpha)*mu_c
    w1 = alpha*mu_s + (1-alpha)*mu_m
    a0 = (1-x0)*mu_m + x0*w0 - mu_m
    a1 = (1-x1)*mu_m + x1*w1 - mu_m
    aa0[i] = a0*dens
    aa1[i] = a1*dens
    v_n += a0*dens
    x_n += x0*dens
    oa_n += x0*alpha*dens
    v_t += a1*dens
    x_t += x1*dens
    oa_t += x1*alpha*dens
    c_t += x1*alpha*dens

In [34]:
# set up the merger policy
    if alpha>alpha_h:
        v_h += a1*dens
        x_h += x1*dens
        oa_h += x1*alpha*dens
        c_h += x1*alpha*dens
    else:
        v_h += a0*dens
        x_h += x0*dens
        oa_h += x0*alpha*dens
    if alpha>alpha_p:
        v_p += a1*dens
        x_p += x1*dens
        oa_p += x1*alpha*dens
        c_p += x1*alpha*dens
    else:
        v_p += a0*dens
        x_p += x0*dens
        oa_p += x0*alpha*dens
    if a1>a0:
        v_o += a1*dens
        x_o += x1*dens
        oa_o += x1*alpha*dens
        c_o += x1*alpha*dens
        if flag==0:
            alphastar = alpha
            flag=1
    else:
        v_o += a0*dens
        x_o += x0*dens
        oa_o += x0*alpha*dens


In [35]:
 print(('%.5f,'*6+'%.5f') % (alpha,x0*100,(w0-100)*1000,a0*1000000,x1*100,(w1-100)*1000,a1*1000000),file=f2)

In [36]:
print(test,alpha)
print()
print(alpha_h,alphastar)
print()
mult1 = 1000000
mult2 = 100
mult3 = 10000
print('Welfare per startup (\$000)',file=f1)
print('& %.3f '*5 % (v_n*mult1, v_h*mult1, v_p*mult1, v_o*mult1, v_t*mult1),sep=',',end='\n',file=f1)
print(' \snl',file=f1)
print('Blocked mergers (\%)',file=f1)
print('& %.3f '*5 % (0*mult2, np.exp(-alpha_h/alphabar)*mult2, np.exp(-.5/alphabar)*mult2, np.exp(-alphastar/alphabar)*mult2, 1*mult2),sep=',',end='\n',file=f1)
print(' \snl',file=f1)
print('\# successful startups',file=f1)
print('& %.3f '*5 % (x_n*mult3, x_h*mult3, x_p*mult3, x_o*mult3, x_t*mult3),sep=',',end='\n',file=f1)
print(' \snl',file=f1)
print('$\E(\\alpha)$ successful startups (\%)',file=f1)
print('& %.3f '*5 % (oa_n/x_n*mult2, oa_h/x_h*mult2, oa_p/x_p*mult2, oa_o/x_o*mult2, oa_t/x_t*mult2),sep=',',end='\n',file=f1)
print(' \snl',file=f1)
print('\# competitors',file=f1)
print('& %.5f '*5 % (c_n*mult3, c_h*mult3, c_p*mult3, c_o*mult3, c_t*mult3),sep=',',end='\n',file=f1)
f1.close()
f2.close()

1.010033331070502 1.0

0.06913539905573864 1.0



In [37]:
f1=open('alpha0.csv', 'w+')
print('alpha,v',file=f1)
v = np.zeros(nobs)
for i in range(0,nobs):
    for k in range(0,i):
        v[i] += aa0[k]
    for k in range(i,nobs):
        v[i] += aa1[k]
for i in range(0,nobs):
    alpha = i*grid
    print(alpha,v[i]*1000,sep=',',end='\n',file=f1)
f1.close()